In [2]:
import os, json, requests
from questdb.ingress import Sender, TimestampNanos
from polymarket.src.config.enum import QuestDBPartitionBy
from polymarket.src.db.queries import build_drop_table_query, build_create_table_query
from polymarket.src.db.run_query_http import execute_https_query

In [3]:
host = "localhost"
port = "9000"
conf = f"http::addr={host}:{port};username=admin;password=quest;"

SCHEMA = """
    timestamp TIMESTAMP,
    symbol SYMBOL,
    price DOUBLE,
    amount DOUBLE
"""

In [4]:
drop_query = build_drop_table_query(table_name="trades")
response_drop_table = execute_https_query(host=host, port=port, query=drop_query)
create_query = build_create_table_query(table_name="trades", schema=SCHEMA, partition_by=QuestDBPartitionBy.DAY)
response_create_table = execute_https_query(host=host, port=port, query=create_query)

In [5]:
from questdb.ingress import Sender, IngressError, TimestampNanos
import sys
import datetime


def example():
    try:
        conf = (
            'http::addr=localhost:9000;'
            'username=admin;password=quest;'
            'auto_flush_rows=100;auto_flush_interval=1000;')
        with Sender.from_conf(conf) as sender:
            # Record with provided designated timestamp (using the 'at' param)
            # Notice the designated timestamp is expected in Nanoseconds,
            # but timestamps in other columns are expected in Microseconds.
            # You can use the TimestampNanos or TimestampMicros classes,
            # or you can just pass a datetime object
            sender.row(
                'trades',
                symbols={
                    'symbol': 'ETH-USD',
                    'side': 'sell'},
                columns={
                    'price': 2615.54,
                    'amount': 0.00044},
                at=datetime.datetime(
                    2022, 3, 8, 18, 53, 57, 609765,
                    tzinfo=datetime.timezone.utc))

            # You can call `sender.row` multiple times inside the same `with`
            # block. The client will buffer the rows and send them in batches.

            # You can flush manually at any point.
            sender.flush()

            # If you don't flush manually, the client will flush automatically
            # when a row is added and either:
            #   * The buffer contains 75000 rows (if HTTP) or 600 rows (if TCP)
            #   * The last flush was more than 1000ms ago.
            # Auto-flushing can be customized via the `auto_flush_..` params.

        # Any remaining pending rows will be sent when the `with` block ends.

    except IngressError as e:
        sys.stderr.write(f'Got error: {e}\n')



example()